In [1]:
!pip install tensorflow
!pip install matplotlib
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt
from tensorflow import keras
import keras.layers
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,recall_score,precision_score
import numpy as np

In [2]:
import zipfile
import os
zip_file_path = './mrismall800.zip'
extract_dir = './mrismall800'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(os.listdir(extract_dir))

['mri']


In [3]:
IMAGE_SIZE=650
BATCH_SIZE=32

In [4]:
import os
print(os.listdir('./mrismall800/mri/dataset'))

['Normal', 'Stroke']


In [5]:
df=tf.keras.preprocessing.image_dataset_from_directory(
    "mrismall800/mri/dataset",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 800 files belonging to 2 classes.


In [6]:
class_names=df.class_names
class_names

['Normal', 'Stroke']

In [7]:
image_count=0
for images, labels in df:
    image_count += images.shape[0]

In [8]:
image_count

800

In [9]:
def  total_images(dataset):
    count = 0
    for _ in dataset:
        count += 1
    return count

In [10]:
def split(df,train_split=0.75,test_split=0.25,shuffle=True,shuffle_size=5000):
  df_shape= total_images(df.unbatch())
  df = df.unbatch().batch(1)
  if shuffle:
    df=df.shuffle(shuffle_size,seed=50)
  train_size=int(train_split*df_shape)
  test_size=int(test_split*df_shape)
  train_df=df.take(train_size)
  test_df=df.skip(train_size).take(test_size)
  return train_df,test_df


In [11]:
train_imgs,test_imgs=split(df)

In [12]:
resize_rescale=tf.keras.Sequential([
    layers.Resizing(650,650),
    layers.Rescaling(1.0/255)
])
data_enhance = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3)
])

In [19]:
def eleven_layer_cnn_softmax_kernel5():
  model = keras.models.Sequential([
      resize_rescale,
    data_enhance,
    keras.layers.Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(650,650,3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=128, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
      ])
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


In [20]:
train_labels = np.concatenate([y.numpy() for _, y in train_imgs], axis=0)
test_labels = np.concatenate([y.numpy() for _, y in test_imgs], axis=0)

In [21]:
model1=eleven_layer_cnn_softmax_kernel5()
model1.fit(train_imgs,epochs=5)
y_pred_probs = model1.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model 1")
print(accuracy)
print(f1)
print(precision)
print(recall)

Epoch 1/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 783s 1s/step - accuracy: 0.7460 - loss: 7.2398
Epoch 2/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 736s 1s/step - accuracy: 0.7286 - loss: 0.6071
Epoch 3/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 730s 1s/step - accuracy: 0.7514 - loss: 0.7828
Epoch 4/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 728s 1s/step - accuracy: 0.7557 - loss: 0.5669
Epoch 5/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 711s 1s/step - accuracy: 0.7526 - loss: 0.5624
200/200 ━━━━━━━━━━━━━━━━━━━━ 94s 459ms/step
Model 1
0.765
0.0
0.0
0.0


In [22]:
def nine_layer_cnn_softmax_kernel5():
  model = keras.models.Sequential([
      resize_rescale,
    data_enhance,
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu', input_shape=(650,650,3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=128, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
      ])
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [23]:
model2=nine_layer_cnn_softmax_kernel5()
model2.fit(train_imgs,epochs=5)
y_pred_probs = model2.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model 2")
print(accuracy)
print(f1)
print(precision)
print(recall)

Epoch 1/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 2242s 4s/step - accuracy: 0.7541 - loss: 3.0601
Epoch 2/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 2401s 4s/step - accuracy: 0.7651 - loss: 0.5639
Epoch 3/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 2968s 5s/step - accuracy: 0.7380 - loss: 6.6904
Epoch 4/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 2385s 4s/step - accuracy: 0.7499 - loss: 0.5849
Epoch 5/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 2536s 4s/step - accuracy: 0.7513 - loss: 0.5685
200/200 ━━━━━━━━━━━━━━━━━━━━ 303s 1s/step
Model 2
0.765
0.0
0.0
0.0


In [24]:
def densenet121():
    base_model=tf.keras.applications.DenseNet121(input_shape=(650,650,3),
                                                include_top=False,
                                                weights='imagenet')
    base_model.trainable=False
    model=tf.keras.models.Sequential([
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(512,activation="relu"),
        keras.layers.Dense(2,activation='softmax')
    ])
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

In [25]:
model3=densenet121()
model3.fit(train_imgs,epochs=5)
y_pred_probs = model3.predict(test_imgs.map(lambda x, y: x))
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
print("Model")
print(accuracy)
print(f1)
print(precision)
print(recall)

Epoch 1/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 936s 2s/step - accuracy: 0.6701 - loss: 2.4029
Epoch 2/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 932s 2s/step - accuracy: 0.7629 - loss: 0.4871
Epoch 3/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 944s 2s/step - accuracy: 0.7996 - loss: 0.4561
Epoch 4/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 895s 1s/step - accuracy: 0.7935 - loss: 0.4647
Epoch 5/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 889s 1s/step - accuracy: 0.7869 - loss: 0.4322
200/200 ━━━━━━━━━━━━━━━━━━━━ 292s 1s/step
Model
0.755
0.07547169811320754
0.3333333333333333
0.0425531914893617


In [26]:
from sklearn.ensemble import VotingClassifier

In [35]:

def soft_voting(models, X):
    y_pred_probs = [model.predict(X) for model in models]
    avg_pred_probs = np.mean(y_pred_probs, axis=0)
    y_pred = np.argmax(avg_pred_probs, axis=1)
    return y_pred, avg_pred_probs

models = [model1, model2, model3]
y_pred, y_pred_probs = soft_voting(models, test_imgs.map(lambda x, y: x))
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred, average='weighted')
precision = precision_score(test_labels, y_pred, average='weighted')
recall = recall_score(test_labels, y_pred, average='weighted')
print("Model4")
print(accuracy)
print(f1)
print(precision)
print( recall)


200/200 ━━━━━━━━━━━━━━━━━━━━ 89s 413ms/step
200/200 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step
200/200 ━━━━━━━━━━━━━━━━━━━━ 299s 1s/step
Model4
0.765
0.6631444759206799
0.585225
0.765


In [72]:
test_labels,y_pred_probs

(array([0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
        1, 1]),
 array([[0.8268256 , 0.17317437],
        [0.82183534, 0.17816468],
        [0.6084627 , 0.39153728],
        [0.8250029 , 0.17499705],
        [0.8103247 , 0.18967526],
        [0.82123965, 0.17876035],
        [0.7839847 , 0.21601526],
        [0.8248048 , 0.17519517],
        [0.58920646, 0.41079357],
        [0.8

In [41]:
import pickle


In [33]:
for i, model in enumerate(models, start=1):
    with open(f'model{i}.pkl', 'wb') as model_file:
        pickle.dump(model, model_file)

In [ ]:
# ! pip install -q gradio
# with open('model.pkl','rb') as file:
#   model=pickle.load(file)

In [102]:

def soft_voting(models, X):
    y_pred_probs = [model.predict(X) for model in models]
    #print(f"Predictions from each model: {[prob.shape for prob in y_pred_probs]}")
    avg_pred_probs = np.mean(y_pred_probs, axis=0)    
    #print(f"Average predicted probabilities: {avg_pred_probs}")
    y_pred = np.argmax(avg_pred_probs, axis=1)
    return y_pred, avg_pred_probs

In [103]:
def image_preprocess(image_path):
  img=Image.open(image_path).convert('RGB')
  img=img.resize((650,650))
  img=np.array(img)
  img=img/255.0
  return img
def prediction(image_path,location):
  img=image_preprocess(image_path)
  img=np.expand_dims(img,axis=0)
  models = [model1, model2, model3]
  y_pred, y_pred_probs = soft_voting(models, img)
  pred_class=y_pred[0]
  print(f"Final prediction: {y_pred}")
  print(f"Predicted probabilities: {y_pred_probs}") 
#   pred_class=np.argmax(pred[0])
  pred_msg=""
  maps=""
  if pred_class==1:
   pred_msg="Brain Stroke Detected"
   maps_url=f"https://www.google.com/maps/search/hospitals+near+{location}"
   maps=f"Find nearby hospitals: [Google Maps]({maps_url})"   
  else:
   pred_msg= "No brain stroke detected"
   maps="Visiting a Hospital is not required"
    
  return pred_msg,maps
  

In [104]:
img1="normal3.jpg"
img2="normal4.jpg"
img3="stroke4.jpg"
img4="97 (1).jpg"
img5="97 (10).jpg"
img6="97 (11).jpg"
img7="97 (12).jpg"
img8="97 (13).jpg"
img9="97 (15).jpg"
img10="97 (18).jpg"
img11="97 (2).jpg"
img12="97 (24).jpg"
img13="97 (26).jpg"
img14="97 (28).jpg"
img15="97 (3).jpg"
img16="97 (32).jpg"
img17="97 (34).jpg"
img18="97 (35).jpg"
img19="97 (36).jpg"
img20="97 (38).jpg"

pred_class1=prediction(img1,"malakpet")
pred_class2=prediction(img2,"malakpet")
pred_class3=prediction(img3,"gandhinagar")
print((pred_class1))
print((pred_class2))
print((pred_class3))
pred_class1=prediction(img4,"malakpet")
pred_class2=prediction(img5,"malakpet")
pred_class3=prediction(img6,"gandhinagar")
print((pred_class1))
print((pred_class2))
print((pred_class3))
pred_class1=prediction(img7,"malakpet")
pred_class2=prediction(img8,"malakpet")
pred_class3=prediction(img9,"gandhinagar")
print((pred_class1))
print((pred_class2))
print((pred_class3))
pred_class1=prediction(img10,"malakpet")
pred_class2=prediction(img11,"malakpet")
pred_class3=prediction(img12,"gandhinagar")
print((pred_class1))
print((pred_class2))
print((pred_class3))
pred_class1=prediction(img13,"malakpet")
pred_class2=prediction(img14,"malakpet")
pred_class3=prediction(img15,"gandhinagar")
print((pred_class1))
print((pred_class2))
print((pred_class3))
pred_class1=prediction(img16,"malakpet")
pred_class2=prediction(img17,"malakpet")
pred_class3=prediction(img18,"gandhinagar")
print((pred_class1))
print((pred_class2))
print((pred_class3))
pred_class1=prediction(img19,"malakpet")
pred_class2=prediction(img20,"malakpet")
print((pred_class1))
print((pred_class2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Final prediction: [0]
Predicted probabilities: [[0.67974216 0.3202578 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Final prediction: [0]
Predicted probabilities: [[0.6806063  0.31939372]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Final prediction: [0]
Predicted probabilities: [[0.67938966 0.3206104 ]]
('No brain stroke detected', 'Visiting a Hospital is not required')
('No brain stroke detected', 'Visiting a Hospital is not required')
('No brain stroke detected', 'Visiting a Hospital is not required')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Final prediction: [0]
Predicted probabilities: [[0.69251394 0.30748612]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
1/1 ━━━━━━━━━━━━━━━━━

In [101]:
img1="58 (10).jpg"
pred_class1=prediction(img1,"malakpet")
print(pred_class1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predictions from each model: [(1, 2), (1, 2), (1, 2)]
Average predicted probabilities: [[0.7297139  0.27028608]]
Final prediction: [0]
Predicted probabilities: [[0.7297139  0.27028608]]
('No brain stroke detected', 'Visiting a Hospital is not required')


In [87]:
!pip install -q gradio
import gradio as gr

In [105]:
interface = gr.Interface(
    fn=prediction,
    inputs=[
gr.Image(type="filepath"),
gr.Textbox(label="Location")],
    outputs=[gr.Textbox(label="Prediction By Model"),
            gr.Markdown(label="Maps")],
    title="Brain Stroke Prediction with  MRI Scans using Ensemble Machine Learning"
)
interface.launch(share=True)



Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://b9d516927703300a7f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
